In [ ]:
!pip uninstall transformers
!pip install transformers==4.31.0

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.41.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import tensorflow as tf

In [ ]:
df = pd.read_csv('../dataset/topic_validation.csv')

df = df[df['summary'].notnull()]
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df = train_df.rename(columns={'Description': 'source_text', 'summary': 'target_text'})
val_df = val_df.rename(columns={'Description': 'source_text', 'summary': 'target_text'})


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
def tokenize_data(data, max_source_length, max_target_length):
    source_texts = data['source_text'].tolist()
    target_texts = data['target_text'].tolist()

    inputs = tokenizer(source_texts, max_length=max_source_length, padding='max_length', truncation=True, return_tensors='tf')
    targets = tokenizer(target_texts, max_length=max_target_length, padding='max_length', truncation=True, return_tensors='tf')

    return inputs, targets


In [ ]:
train_inputs, train_targets = tokenize_data(train_df, max_source_length=512, max_target_length=128)
val_inputs, val_targets = tokenize_data(val_df, max_source_length=512, max_target_length=128)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_inputs),
    dict(labels=train_targets['input_ids'])
)).shuffle(len(train_df)).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_inputs),
    dict(labels=val_targets['input_ids'])
)).batch(8)


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer)

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

Epoch 1/3
49/49 [==============================] - 74s 492ms/step - loss: 1.9472 - val_loss: 1.4011
Epoch 2/3
49/49 [==============================] - 21s 424ms/step - loss: 1.4680 - val_loss: 1.2478
Epoch 3/3
49/49 [==============================] - 21s 429ms/step - loss: 1.3400 - val_loss: 1.1597


In [ ]:
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="tf", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:


text_to_summarize = """
Course Introduction:
Selamat datang di matakuliah Komunikasi Data dan Jaringan Komputer! Matakuliah ini akan membahas tentang bagaimana data dikirimkan dan diterima melalui jaringan komputer. Kamu akan belajar tentang berbagai metode komunikasi data, protokol jaringan, dan bagaimana cara mengatur dan mengoptimalkan jaringan komputer.

Objective:
Tujuan dari matakuliah ini adalah agar kamu memahami dasar-dasar komunikasi data dan jaringan komputer, serta mampu mengimplementasikan pengaturan jaringan yang efisien dan aman. Dengan menguasai matakuliah ini, diharapkan kamu dapat menjalankan dan mengelola jaringan komputer dengan baik di berbagai industri dan organisasi.
"""
summary = summarize(text_to_summarize)
print(summary)

matakuliah Komunikasi Data dan Jaringan Komputer. membahas tentang berbagai metode komunikasi data, protokol jaringan, dan bagaimana cara mengatur dan mengoptimalkan jaringan komputer.


In [ ]:
model.save_pretrained("t5_text_summarization_model_tf")
tokenizer.save_pretrained("t5_text_summarization_model_tf")

('t5_text_summarization_model_tf/tokenizer_config.json',
 't5_text_summarization_model_tf/special_tokens_map.json',
 't5_text_summarization_model_tf/spiece.model',
 't5_text_summarization_model_tf/added_tokens.json')

In [ ]:
model = TFT5ForConditionalGeneration.from_pretrained("t5_text_summarization_model_tf")
tokenizer = T5Tokenizer.from_pretrained("t5_text_summarization_model_tf")


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5_text_summarization_model_tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
!du -sh /content/t5_text_summarization_model_tf

358M	/content/t5_text_summarization_model_tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!zip -r models_summary.zip /content/t5_text_summarization_model_tf

  adding: content/t5_text_summarization_model_tf/ (stored 0%)
  adding: content/t5_text_summarization_model_tf/tokenizer_config.json (deflated 83%)
  adding: content/t5_text_summarization_model_tf/generation_config.json (deflated 29%)
  adding: content/t5_text_summarization_model_tf/spiece.model (deflated 48%)
  adding: content/t5_text_summarization_model_tf/config.json (deflated 63%)
  adding: content/t5_text_summarization_model_tf/tf_model.h5 (deflated 25%)
  adding: content/t5_text_summarization_model_tf/special_tokens_map.json (deflated 86%)


In [ ]:
!cp models_summary.zip /content/drive/My\ Drive/